# **9. NLP 세션 과제**

- 이번 과제는 BPE 토크나이저를 직접 구현해보고, 훈련시켜보는 것입니다.

- **각 셀의 실행 결과물을 같이** 저장해서, 드라이브에 업로드 된 과제 명세에 적혀있는 제출 방식을 참고하여 본 노트북 파일을 제출해주세요. </br></br>




#### **주의사항!**

- BPE를 구현하는 코드는 조금만 검색해도 쉽게 찾을 수 있습니다. (유명하니까요...) 하지만 해당 코드를 먼저 찾아보는 것은 추천하지 않습니다.

- 본 과제의 목적은 구현 능력을 기르는 데 있습니다. 이론을 실제 코드로 구현하는 능력이 있고, 없고는 굉장히 큰 차이입니다!!

- 따라서, 본 과제를 수행할 때, **BPE의 이론을 먼저
충분히 복습**하고, **다른 샘플 코드를 보지 않고** 각각의 기능을 어떻게 구현할지 충분히 고민해보시기 바랍니다!

- 인터넷에 샘플 코드가 많다는 건 여러분들도 충분히 하실 수 있다는 의미니까요!!!

## 1. BPE Tokenizer 구현하기 (70점, 각 세부 파트별 부분점수 있음)

  **BPE Tokenizer**를 구현하기 위해서, 크게 다음 **세가지** 부분으로 쪼갤 수 있습니다.

```
  1) 전체 corpus를 공백으로 분리하고, {단어:빈도수} 형태의 vocabulary 사전을 구성하는 부분

  2) 구성된 vocabulary 사전을 순회하면서, 사전 내 character 토큰 및 각각의 등장 횟수를 반환해 주는 부분

  3) text에서 가장 자주 등장한 pair을 연결해주는 부분
```

본 과제에서 하실 일은 각각의 함수 코드를 직접 작성해보고, 각각의 함수를 모두 합쳐 최종적으로 BPE Tokenizer를 구현하는 것입니다!

**Hint**: Collections 패키지의 defaultdict 클래스의 활용법을 찾아보시면 좀 더 손쉽게 구현할 수 있습니다.

In [56]:
from collections import defaultdict

여러분이 작성한 코드가 맞는지 스스로 감을 잡으실 수 있도록, 각 함수 별로 샘플 corpus를 함수에 넣었을 때 나와야 하는 결과를 함께 제시하도록 하겠습니다. 샘플 corpus는 다음과 같습니다.

In [57]:
sample_corpus =  ['YBAGTA YBECTA YBIGTA YBOGTA YBUGTA',
                  'I love YBIGTA',
                  'I like YBIGTA',
                  'what is YBAGTA']

### 1) 사전만들기 함수 (20점)

 첫번째 함수는 corpus가 입력으로 주어졌을 때, 이에 기반하여 Vocabulary 사전을 구축하는 함수입니다. 이때, input/output 형태는 다음과 같습니다.


  - `input` : 여러 개의 문자열로 이루어진 리스트 (sample corpus와 형태 동일)
  - `output` : 여러 개의 단어:빈도수 쌍으로 이루어진 딕셔너리 (예시: `{단어1:빈도수1, 단어2:빈도수2, ... }` )



**TODO**:
  input 리스트 안의 문자열들에 대해서, 공백을 기준으로 분리한 각각의 단어와 그 빈도수를 딕셔너리 형태로 리턴하는 함수를 정의하세요.

 이때, 추후 두번째, 세번째 함수 기능 구현의 편의성을 위해, output 딕셔너리 내 단어 형식을 다음과 같이 맞춰주세요,
- 예시: `'Y B I G T A </w>'` -> 글자 사이 공백 추가, 마지막에 postfix 토큰  `</w>` 추가

In [58]:
from collections import Counter

def 사전만들기(corpus):

  ##### TODO #####
  # 함수 안을 채워주세요 #
  # 이미 작성된 코드는 지우시면 안됩니다! #

  word_dict = Counter([word for line in corpus for word in line.split()])
  word_formatted = [" ".join(word) + " </w>" for word in word_dict.keys()]
  vocab = dict(zip(word_formatted, word_dict.values()))

  return vocab

위에서 정의한 `사전만들기` 함수를 sample_corpus에 적용시 다음과 같은 결과가 리턴되어야 합니다.


- 입력:
```
vocab = 사전만들기(sample_corpus)
vocab
```

- 결과:
```
{'Y B A G T A </w>': 2,
 'Y B E C T A </w>': 1,
 'Y B I G T A </w>': 3,
 'Y B O G T A </w>': 1,
 'Y B U G T A </w>': 1,
 'I </w>': 2,
 'l o v e </w>': 1,
 'l i k e </w>': 1,
 'w h a t </w>': 1,
 'i s </w>': 1}
 ```

In [59]:
# 다음 코드를 실행한 결과값을 위의 결과와 비교해서 함수를 잘 작성했는지 확인해보세요!
vocab = 사전만들기(sample_corpus)
vocab

{'Y B A G T A </w>': 2,
 'Y B E C T A </w>': 1,
 'Y B I G T A </w>': 3,
 'Y B O G T A </w>': 1,
 'Y B U G T A </w>': 1,
 'I </w>': 2,
 'l o v e </w>': 1,
 'l i k e </w>': 1,
 'w h a t </w>': 1,
 'i s </w>': 1}

### 2) 토큰등장횟수 함수 (20점)

다음으로는, `사전만들기`함수로 얻은 vocab 딕셔너리를 입력을 받아서, 사전에 포함된 글자 각각의 등장 횟수를 반환해주는 함수를 구현해 봅시다.

인풋, 아웃풋 형태는 다음과 같습니다.

- `input` : `사전만들기`함수의 output
- `output`: {글자:빈도수} 구조의 tokens 딕셔너리,
    예) {'Y':4, 'B':8, .... }

In [60]:
def 토큰등장횟수(vocab):

  ##### TODO #####
  # 함수 안을 채워주세요 #
  # 이미 작성된 코드는 지우시면 안됩니다! #
  word_list = [k for k, v in vocab.items() for _ in range(v)]
  result = Counter([token for word in word_list for token in word.split()])
  return dict(result)

첫번째로 정의한 `사전만들기`함수에서 반환된 vocab을 `토큰등장횟수`에 넣었을때 반환되는 결과는 다음과 같아야 합니다.

```
{'Y': 8,
 'B': 8,
 'A': 10,
 'G': 7,
 'T': 8,
 '</w>': 14,
 'E': 1,
 'C': 1,
 'I': 5,
 'O': 1,
 'U': 1,
 'l': 2,
 'o': 1,
 'v': 1,
 'e': 2,
 'i': 2,
 'k': 1,
 'w': 1,
 'h': 1,
 'a': 1,
 't': 1,
 's': 1}
 ```

 위와 같은 결과가 나오는지, 다음 셀을 실행하여 확인해볼 수 있습니다.

In [61]:
# 다음 코드를 실행한 결과값을 위의 결과와 비교해서 함수를 잘 작성했는지 확인해보세요!
result = 토큰등장횟수(vocab)
result

{'Y': 8,
 'B': 8,
 'A': 10,
 'G': 7,
 'T': 8,
 '</w>': 14,
 'E': 1,
 'C': 1,
 'I': 5,
 'O': 1,
 'U': 1,
 'l': 2,
 'o': 1,
 'v': 1,
 'e': 2,
 'i': 2,
 'k': 1,
 'w': 1,
 'h': 1,
 'a': 1,
 't': 1,
 's': 1}

### 3) 페어합치기 함수 (30점)

마지막 함수입니다. 마지막으로, corpus에서 가장 자주 등장한 페어를 엮어 새롭게 vocab에 추가하는 기능을 구현해봅시다.

이를 구현하기 위해서, 다시 다음 두 단계로 쪼갤 수 있습니다.

1) `페어등장횟수` (15점): 각 페어가 등장한 횟수를 세어 딕셔너리 형태로 반환
- `input` : vocab
- `output`: {(글자1,글자2):등장횟수, (글자2,글자3):등장횟수,...} 형식의 딕셔너리
 -> 아래 샘플 결과 형태를 확인하세요.


2) `페어합치기` (15점):
-   `input`: vocab, 가장 자주 등장한 Tuple 형태의 pair
- `output`: 새롭게 업데이트한 tokens 사전


먼저, `페어등장횟수`함수부터 구현해 봅시다.


In [62]:
def 페어등장횟수(vocab):

  ##### TODO #####
  # 함수 안을 채워주세요 #
  # 이미 작성된 코드는 지우시면 안됩니다! #
  pairs = defaultdict(int)
  for word, freq in vocab.items():
    tokens = word.split()
    for i in range(len(tokens) - 1):
      pair = (tokens[i], tokens[i + 1])
      pairs[pair] += freq

  return pairs

위 함수에 우리의 예시를 넣어보면 다음과 같은 결과가 나와야 합니다.

```
{('Y', 'B'): 8,
 ('B', 'A'): 2,
 ('A', 'G'): 2,
 ('G', 'T'): 7,
 ('T', 'A'): 8,
 ('A', '</w>'): 8,
 ('B', 'E'): 1,
 ('E', 'C'): 1,
 ('C', 'T'): 1,
 ('B', 'I'): 3,
 ('I', 'G'): 3,
 ('B', 'O'): 1,
 ('O', 'G'): 1,
 ('B', 'U'): 1,
 ('U', 'G'): 1,
 ('I', '</w>'): 2,
 ('l', 'o'): 1,
 ('o', 'v'): 1,
 ('v', 'e'): 1,
 ('e', '</w>'): 2,
 ('l', 'i'): 1,
 ('i', 'k'): 1,
 ('k', 'e'): 1,
 ('w', 'h'): 1,
 ('h', 'a'): 1,
 ('a', 't'): 1,
 ('t', '</w>'): 1,
 ('i', 's'): 1,
 ('s', '</w>'): 1}
 ```

 위와 같이, 튜플로 묶인 두 개의 토큰 페어와, 각 페어별 등장횟수로 이루어진 딕셔너리가 반환되어야 합니다. 아래 코드를 실행해 결과값이 같은지 확인해보세요.

In [63]:
# 다음 코드를 실행한 결과값을 위의 결과와 비교해서 함수를 잘 작성했는지 확인해보세요!
pairs = 페어등장횟수(vocab)
pairs

defaultdict(int,
            {('Y', 'B'): 8,
             ('B', 'A'): 2,
             ('A', 'G'): 2,
             ('G', 'T'): 7,
             ('T', 'A'): 8,
             ('A', '</w>'): 8,
             ('B', 'E'): 1,
             ('E', 'C'): 1,
             ('C', 'T'): 1,
             ('B', 'I'): 3,
             ('I', 'G'): 3,
             ('B', 'O'): 1,
             ('O', 'G'): 1,
             ('B', 'U'): 1,
             ('U', 'G'): 1,
             ('I', '</w>'): 2,
             ('l', 'o'): 1,
             ('o', 'v'): 1,
             ('v', 'e'): 1,
             ('e', '</w>'): 2,
             ('l', 'i'): 1,
             ('i', 'k'): 1,
             ('k', 'e'): 1,
             ('w', 'h'): 1,
             ('h', 'a'): 1,
             ('a', 't'): 1,
             ('t', '</w>'): 1,
             ('i', 's'): 1,
             ('s', '</w>'): 1})

다음으로, `페어합치기` 함수를 구현해봅시다. `페어합치기` 함수는 `페어등장횟수` 함수로 계산된 페어 빈도수를 바탕으로, BPE 로직에 맞게 vocab을 업데이트합니다.

In [64]:
def 페어합치기(pairs, vocab):

  ##### TODO #####
  # 함수 안을 채워주세요 #
  # 이미 작성된 코드는 지우시면 안됩니다! #
  if not pairs:
    return vocab
  
  max_pair = max(pairs, key=pairs.get)
  new_vocab = {}

  for word in vocab:
    tokens = word.split()
    new_word = word[:]

    for i in range(len(tokens) - 1):
      pair = (tokens[i], tokens[i + 1])
      if pair == max_pair:
        new_word = new_word.replace(" ".join(pair), "".join(pair))

    new_vocab[new_word] = vocab[word]

  return new_vocab

작성한 `페어합치기` 함수에 우리의 예시를 넣으면 다음과 같은 결과를 반환합니다.

```
{'YB A G T A </w>': 2,
 'YB E C T A </w>': 1,
 'YB I G T A </w>': 3,
 'YB O G T A </w>': 1,
 'YB U G T A </w>': 1,
 'I </w>': 2,
 'l o v e </w>': 1,
 'l i k e </w>': 1,
 'w h a t </w>': 1,
 'i s </w>': 1}
 ```

 아래 코드를 실행해 결과값이 같은지 확인해보세요.

In [65]:
# 다음 코드를 실행한 결과값을 위의 결과와 비교해서 함수를 잘 작성했는지 확인해보세요!
vocab = 페어합치기(pairs, vocab)
vocab

{'YB A G T A </w>': 2,
 'YB E C T A </w>': 1,
 'YB I G T A </w>': 3,
 'YB O G T A </w>': 1,
 'YB U G T A </w>': 1,
 'I </w>': 2,
 'l o v e </w>': 1,
 'l i k e </w>': 1,
 'w h a t </w>': 1,
 'i s </w>': 1}

## 2. BPE Tokenizer Train (30점)

이제 BPE 트레이닝을 위한 각 파트 구현을 완료했습니다! **각각의 함수를 합쳐서 BPE Tokenizer의 학습을 수행하는 `학습` 함수를 작성해주세요. (15점)**

 - 1번 파트에서 작성한 함수들은 한번 vocab을 업데이트하는 데 필요한 함수들입니다.
 - `n_iter`인자(int type)를 받아, 입력받은 횟수만큼 vocab을 업데이트하도록 함수를 작성해주세요.

1번 파트에서 모든 함수를 잘 작성했다면, 트레이닝 함수는 간단히 작성하실 수 있습니다.

In [66]:
vocab = 사전만들기(sample_corpus)
# tokens = 토큰등장횟수(vocab)
pairs = 페어등장횟수(vocab)
new_vocab = 페어합치기(pairs, vocab)

In [67]:
def 학습(corpus, n_iter):
  ##### TODO #####
  # 함수 안을 채워주세요 #
  # 이미 작성된 코드는 지우시면 안됩니다! #
  vocab = 사전만들기(corpus)
  for i in range(n_iter):
    pairs = 페어등장횟수(vocab)
    vocab = 페어합치기(pairs, vocab)
  tokens = 토큰등장횟수(vocab)
  return tokens

학습 함수에 sample corpus를 넣고, `n_iter`을 10으로 정한 후 시행한 결과는 다음과 같습니다.

```
{'YBAGTA</w>': 2,
 'YB': 3,
 'E': 1,
 'C': 1,
 'TA</w>': 1,
 'YBIGTA</w>': 3,
 'O': 1,
 'GTA</w>': 2,
 'U': 1,
 'I</w>': 2,
 'l': 2,
 'o': 1,
 'v': 1,
 'e</w>': 2,
 'i': 2,
 'k': 1,
 'w': 1,
 'h': 1,
 'a': 1,
 't': 1,
 '</w>': 2,
 's': 1}
 ```


In [68]:
tokens = 학습(sample_corpus,n_iter=10)
tokens

{'YBAGTA</w>': 2,
 'YB': 3,
 'E': 1,
 'C': 1,
 'TA</w>': 1,
 'YBIGTA</w>': 3,
 'O': 1,
 'GTA</w>': 2,
 'U': 1,
 'I</w>': 2,
 'l': 2,
 'o': 1,
 'v': 1,
 'e</w>': 2,
 'i': 2,
 'k': 1,
 'w': 1,
 'h': 1,
 'a': 1,
 't': 1,
 '</w>': 2,
 's': 1}

**또, 직접 학습을 시켜봅시다. (15점)**


학습용 corpus는 각자 자유롭게 정해주세요. (ex. 노래 가사 등등...)

- 단, 빈도수 기반의 BPE 토크나이저의 효과적인 학습을 위해서, 학습 corpus는 다음 조건을 만족하면 더 좋겠죠?

  - 같은 단어가 여러개 들어가있고 적당히 긴 텍스트

In [69]:
##### TODO #####
# 리스트 안에 train corpus를 자유롭게 채워주세요 #

train_corpus = ['솔직히 말할게 많이 기다려 왔어 너도 그랬을 거라 믿어 오늘이 오길 매일같이 달력을 보면서 솔직히 나에게도 지금 이 순간은 꿈만 같아 너와 함께라 오늘을 위해 꽤 많은 걸 준비해 봤어 All about you and I 다른 건 다 제쳐 두고 Now come with me Take my hand 아름다운 청춘의 한 장 함께 써내려 가자 너와의 추억들로 가득 채울래 (Come on!) 아무 걱정도 하지는 마 나에게 다 맡겨 봐 지금 이 순간이 다시 넘겨볼 수 있는 한 페이지가 될 수 있게 This is our page Our page 솔직히 말할게 지금이 오기까지 마냥 순탄하진 않았지 오늘이 오길 나도 목 빠져라 기다렸어 솔직히 나보다도 네가 몇 배는 더 힘들었을 거라고 믿어 오늘을 위해 그저 견뎌줘서 고마워 All about you and I 다른 건 다 제쳐 두고 Now come with me Take my hand 아름다운 청춘의 한 장 함께 써내려 가자 너와의 추억들로 가득 채울래 (Come on!) 아무 걱정도 하지는 마 나에게 다 맡겨 봐 지금 이 순간이 다시 넘겨볼 수 있는 한 페이지가 될 수 있게 Want you to Come on out and have fun Want us to Have the time of our life Oh 너와의 추억들로 가득 채울래 (Come on!) 아무 걱정도 하지는 마 나에게 다 맡겨 봐 지금 이 순간이 다시 넘겨볼 수 있는 한 페이지가 될 수 있게 This is our page Our page ',
              '지금 이 말이 우리가 다시 시작하자는 건 아냐 그저 너의 남아있던 기억들이 떠올랐을 뿐이야 정말 하루도 빠짐없이 (너는) 사랑한다 말해줬었지 잠들기 전에 또 눈 뜨자마자 말해주던 너 생각이 나 말해보는 거야 예뻤어 날 바라봐 주던 그 눈빛 날 불러주던 그 목소리 다 - 다 - 그 모든 게 내겐 예뻤어 더 바랄게 없는듯한 느낌 오직 너만이 주던 순간들 다 - 다 - 지났지만 넌 너무 예뻤어 너도 이제는 나와의 기억이 추억이 되었을 거야 너에게는 어떤 말을 해도 다 지나간 일일 거야 정말 한번도 빠짐없이 (너는) 나를 먼저 생각해줬어 아무 일 아니어도 미안해 고마워 해주던 너 생각이 나 말해보는 거야 예뻤어 날 바라봐 주던 그 눈빛 날 불러주던 그 목소리 다 - 다 - 그 모든 게 내겐 예뻤어 더 바랄게 없는듯한 느낌 오직 너만이 주던 순간들 다 - 다 - 지났지만 넌 너무 예뻤어 아직도 가끔 네 생각이 나 어렵게 전화를 걸어볼까 생각이 들 때도 많지만 Baby I know it\'s already over 아무리 원해도 너는 이제 이미 끝나버린 지난날의 한편의 영화였었단 걸 난 알아 마지막 날 바라봐 주던 그 눈빛 잘 지내라던 그 목소리 다 - 다 - 그마저도 내겐 예뻤어 내게 보여준 눈물까지 너와 가졌던 순간들은 다 - 다 - 지났지만 넌 너무 예뻤어 ',
              'Na na na na na na na na na na Na na na na na na na na na na 내가 살아왔던 세상이 너로 인해 뒤집어져 뒤바뀌어 right is left Left is right 나만의 모래성 너란 파도를 맞고 무너지기 시작해 나 어떡해 help me 너 때문에 I\'m losing control It\'s a sweet chaos 니가 등장하면서부터 내 삶과 꿈 미래 그 모든 게 바뀌어 근데 기다려져 내일이 변해가는 매일이 좋아 미칠 정도야 It\'s a sweet chaos oh oh 널 알아가면 갈수록 내 취향과 패턴 그 모든 게 바뀌어 그런 나를 보면 maybe 바보 같아 보일지 몰라 그래도 좋아 It\'s a sweet chaos 모든 게 불안해 But I like 바쁜 거리 속의 사람들 발걸음처럼 계속 정신없이 얽히고 설킨 채 복잡한 머리 자꾸 틀어지는 과거의 내가 쌓아온 계획 망가지는 그 공간을 네가 메우고 있어 무질서 그 안에서 꽃이 피어나 It\'s a sweet chaos oh oh 니가 등장하면서부터 내 삶과 꿈 미래 그 모든 게 바뀌어 근데 기다려져 내일이 변해가는 매일이 좋아 미칠 정도야 sweet chaos 내 안의 나침반은 지금 정신없이 돌아가지만 너의 미소 잠깐이면 난 다 괜찮아져 it\'s a sweet chaos 니가 등장하면서부터 내 삶과 꿈 미래 그 모든 게 바뀌어 근데 기다려져 내일이 변해가는 매일이 좋아 미칠 정도야 It\'s a sweet chaos oh oh 널 알아가면 갈수록 내 취향과 패턴 그 모든 게 바뀌어 그런 나를 보면 maybe 바보 같아 보일지 몰라 그래도 좋아 It\'s a sweet chaos 모든 게 불안해 But I like ',
              ]

In [70]:
##### TODO #####
# 여러분의 train_corpus를 구현한 BPE 토크나이저 학습 함수에 넣고, 결과값을 프린트하세요
tokens = 학습(train_corpus, n_iter=100)
print(tokens)

{'솔직히</w>': 4, '말': 6, '할': 2, '게</w>': 21, '많': 3, '이</w>': 21, '기다려': 4, '</w>': 152, '왔': 2, '어</w>': 7, '너': 19, '도</w>': 18, '그': 8, '랬': 1, '을</w>': 9, '거': 4, '라</w>': 5, '믿': 2, '오늘': 4, '오': 5, '길': 2, '매': 4, '일': 6, '같': 4, '달': 1, '력': 1, '보': 11, '면서': 4, '나에': 4, '게': 2, '지금</w>': 6, '순간': 7, '은</w>': 4, '꿈': 4, '만</w>': 1, '아</w>': 4, '너와': 5, '함': 3, '께': 3, '위': 2, '해</w>': 9, '꽤': 1, '걸': 4, '준': 2, '비': 1, '봤': 1, 'A': 2, 'l': 12, 'a': 10, 'b': 5, 'ou': 9, 't</w>': 11, 'y': 5, 'an': 5, 'd</w>': 5, 'I</w>': 5, '다': 5, '른': 2, '건': 3, '다</w>': 20, '제': 4, '쳐': 2, '두': 2, '고</w>': 6, 'N': 4, 'o': 9, 'w': 5, 'c': 3, 'ome</w>': 2, 'i': 11, 't': 7, 'h</w>': 3, 'me</w>': 4, 'T': 4, 'ke</w>': 4, 'm': 5, 'y</w>': 4, 'ha': 3, 'n': 5, '아': 13, '름': 2, '운': 2, '청': 2, '춘': 2, '의</w>': 14, '한</w>': 8, '장': 5, '써': 2, '내': 12, '려': 2, '가': 14, '자': 6, '추억': 4, '들': 11, '로</w>': 4, '득': 3, '채': 4, '울': 3, '래</w>': 6, '(': 5, 'Come</w>': 4, 'on': 5, '!': 3, ')</w>': 5, '아무</w>': 4, 

## 보너스 질문 ( 추가점수 10점 )
BPE 방식의 한계점은 무엇이고, 이러한 한계점이 왜 나타나는지 고민해보고 그 이유를 자유롭게 설명해주세요.

한계점: BPE는 드물게 사용되는 단어에 대한 표현력이 떨어집니다.  
원인: BPE는 빈도를 기반으로 token들을 병합하는 과정을 통해 dictionary를 구성하기 때문에, 드물게 등장하는 단어들은 병합되지 못하고 여러 개의 token으로 분해되어 표현됩니다. 이는 등장 빈도가 낮은 단어의 표현력을 떨어뜨리는 결과를 야기합니다. 